In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple, Union, List
from enum import Enum, auto
from functools import cached_property
from functools import reduce
from operator import mul
from layout_module import NestedTuple, Atom, Profile, Layout 
from layout_module import nested_tuple_algebra as na
from layout_module import flat_algebra as fa 
from layout_module import layout_algebra as la


In [2]:
x = (3,(2,4))
x_d = (1,(6,12))
y = (3,8)
X = NestedTuple.from_literal(x)
X_d = NestedTuple.from_literal(x_d)
Y = NestedTuple.from_literal(y)
na.refines(X,Y)

True

In [3]:
L = Layout(X, X_d)

In [4]:
V = L.construct_N_complement(96)

In [5]:
V

Layout(shape=(1, 2, 1, 2), stride=(1, 3, 12, 48))

In [6]:
Q = la.concatenate([L,V])
Q


Layout(shape=((3, (2, 4)), (1, 2, 1, 2)), stride=((1, (6, 12)), (1, 3, 12, 48)))

In [7]:
Q.is_compact()

True

In [8]:
Q.is_tractable()

True